In [44]:
#!pip install -U yellowbrick
import pickle
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from numpy import array
from numpy import cov
from numpy.linalg import eig
from factor_analyzer import FactorAnalyzer
import factor_analyzer
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.utils.validation import check_array
from sklearn.base import TransformerMixin
from sklearn.preprocessing import StandardScaler as SklearnStandardScaler

### Read Data

In [1]:
df = pd.read_csv("BASE_DATA.csv",header = None) 


df.columns =['CUSTOMER','SPEND','VISITS','online_gender_perc','basket','age_perc',
             'num_cat_perc','gender_perc','box_style1_perc','box_class_perc',
             'box_style2_perc','income','bkt_sale_perc','bkt_reg_perc',
             'accessory_sales_perc','beauty_sales_perc','men_sales_perc',
             'woman_sales_perc','intimates_sales_perc','kids_sales_perc','woman_footwear_sales_perc']


df = df.set_index('CUSTOMER')
df.head()

### Data Manipulation

In [47]:
features = ['online_gender_perc','basket','age_perc',
             'num_cat_perc','gender_perc','box_style1_perc','box_class_perc',
             'box_style2_perc','income','bkt_sale_perc','bkt_reg_perc',
             'accessory_sales_perc','beauty_sales_perc','men_sales_perc',
             'woman_sales_perc','intimates_sales_perc','kids_sales_perc','woman_footwear_sales_perc']

<a id="3"></a>

In [51]:
class CustomTransformer(TransformerMixin):
    #def __init__(self, with_mean=True, with_std=True): old
    def __init__(self, with_mean=True, with_std=True):
        print("inside transformer function")
     
    def fit(self, X, y=None):
        X[X > 3] = 3
        X[X <-3] =-3

        return self


    def transform(self, X):
        X[X > 3] = 3
        X[X <-3] =-3
        
        return X
      

### K-Means Prediction

In [52]:
#Actual Prediction
X= df[features]
model_pipeline = Pipeline(steps=[('imputer',SimpleImputer(strategy = 'constant',fill_value=0)),
                                  ('standardize', StandardScaler()),
                                ('preprocessor', CustomTransformer()),
                              ('cluster',KMeans(n_clusters=8,init='k-means++', max_iter=80 ,n_init=10, random_state=43))])
model_pipeline.fit(X)

inside transformer function


Pipeline(memory=None,
         steps=[('imputer',
                 SimpleImputer(add_indicator=False, copy=True, fill_value=0,
                               missing_values=nan, strategy='constant',
                               verbose=0)),
                ('standardize',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('preprocessor',
                 <__main__.CustomTransformer object at 0x7f71b557ded0>),
                ('cluster',
                 KMeans(algorithm='auto', copy_x=True, init='k-means++',
                        max_iter=80, n_clusters=8, n_init=10, n_jobs=None,
                        precompute_distances='auto', random_state=43,
                        tol=0.0001, verbose=0))],
         verbose=False)

### Cluster composition

In [2]:
#y = model_pipeline.predict(X)
y=model_pipeline.predict(X)

df_pred = pd.DataFrame(y,index = df.index,columns=['predicted_cluster'])
df_pred = df_pred['predicted_cluster'] + 1   #adjust cluster numbers
df_pred = pd.concat([df, df_pred], axis=1)
df_pred.head()

In [3]:
df_pred.loc[df_pred['predicted_cluster'] == 1, 'Cluster'] = 'Cluster1'
df_pred.loc[df_pred['predicted_cluster'] == 2, 'Cluster'] = 'Cluster2'
df_pred.loc[df_pred['predicted_cluster'] == 3, 'Cluster'] = 'Cluster3'
df_pred.loc[df_pred['predicted_cluster'] == 4, 'Cluster'] = 'Cluster4'
df_pred.loc[df_pred['predicted_cluster'] == 5, 'Cluster'] = 'Cluster5'
df_pred.loc[df_pred['predicted_cluster'] == 6, 'Cluster'] = 'Cluster6'
df_pred.loc[df_pred['predicted_cluster'] == 7, 'Cluster'] = 'Cluster7'
df_pred.loc[df_pred['predicted_cluster'] == 8, 'Cluster'] = 'Cluster8'

customers_per_cluster = df_pred.groupby('Cluster',as_index=True).agg({'age_perc' : ['count']}).rename(columns={'age_perc':'customer'})#COUNT ANY COLUMN
customers_per_cluster

### Save the model

In [57]:
# Save to file in the current working directory
pkl_filename = "pickle_kmeans_pipe_80itr.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(model_pipeline, file)

# Load from file
# with open(pkl_filename, 'rb') as file:
#     pickle_model = pickle.load(file)
    